In [1]:
""" This script tries only using the TuLiP toolbox to solve a multirobot motion planning problem.
Specifically, a field of cells and a number of robots are specified along with the system dynamics,
and constraints are written in monolithic LTL syntax.
"""


# Import the packages that we need
from __future__ import print_function
import numpy as np
from tulip import spec
from tulip import synth
from tulip.transys import machines
import State_encoding
import LTL_encoding

# Workspace parameters
num_rows = 3
num_cols = 3
num_robots = 2
robot_init_pos = np.array([[0, 0], [1, 0], [2, 0]])
# The environment variables and specifications are none
env_vars = set()
env_init = set()
env_prog = set()
env_safe = set()

# Generate robot system dynamics, encoded in sys_safe
#   1. Can move up, down, left, and right
#   2. Can't collide
num_cells = num_rows*num_cols
num_states = num_cells**num_robots
sys_vars = {}
sys_vars['loc'] = (0, num_states - 1)
sys_init_pos = np.array([0, 3])
sys_init = {'loc='+str(State_encoding.array2digit(sys_init_pos, num_cells))}
sys_safe = set()
sys_prog = set()

curr_pos = np.zeros(num_robots, dtype=int)
iter_num_robot = 0
for curr_state in range(num_states):
    LTL = '' # the LTL formula representing the dynamics of this state
    next_state = np.array([], dtype=int)
    if len(np.unique(curr_pos)) == len(curr_pos):
        for next_movement in range(4**num_robots):
            is_valid = True
            temp_curr_pos = np.copy(curr_pos)
            temp_next_movement = next_movement
            for curr_robot in range(num_robots):
                next_move = temp_next_movement % 4
                temp_next_movement //= 4
                # check if this move is valid, i.e., no collision and no out of grid
                if next_move == 0:
                    # move right
                    if curr_pos[curr_robot] % num_cols < num_cols - 1:
                        temp_curr_pos[curr_robot] += 1
                    else:
                        is_valid = False
                        break
                elif next_move == 1:
                    # move up
                    if curr_pos[curr_robot] >= num_cols:
                        temp_curr_pos[curr_robot] -= num_cols
                    else:
                        is_valid = False
                        break
                elif next_move == 2:
                    # move left
                    if curr_pos[curr_robot] % num_cols > 0:
                        temp_curr_pos[curr_robot] -= 1
                    else:
                        is_valid = False
                        break
                elif next_move == 3:
                    # move down
                    if curr_pos[curr_robot] < num_cols * (num_rows - 1):
                        temp_curr_pos[curr_robot] += num_cols
                    else:
                        is_valid = False
                        break
            # check the following circumstance: two robots exchange their positions
            if is_valid:
                for i in range(num_robots):
                    for j in range(i+1, num_robots):
                        if temp_curr_pos[i] == curr_pos[j] and temp_curr_pos[j] == curr_pos[i]:
                            is_valid = False
                            break
                    if not is_valid:
                        break
            if is_valid:
                # check if there is duplicate in the next state
                if len(np.unique(temp_curr_pos)) == len(temp_curr_pos):
                    next_state = np.append(next_state, State_encoding.array2digit(temp_curr_pos, num_cells))

    # It should be the case that there exist some next states
        if len(next_state) > 0:
            LTL = 'loc=' + str(curr_state) + ' -> X (loc=' + str(next_state[0])
            for i in range(1, len(next_state)):
                LTL = LTL + ' || loc=' + str(next_state[i])
            LTL = LTL + ')'
            sys_safe |= {LTL}
    for i in range(num_robots):
        if curr_pos[i] < num_cells - 1:
            curr_pos[i] += 1
            break
        else:
            curr_pos[i] = 0

# Generate inner nad outer logic
# Constraint 1:
# []<> ([6,7,8], 2)
# Constraint 2:
# ([]<> [2], 2)
# Constraint 3:
# []! ([1], 1)
ltl_formula_1 = LTL_encoding.cltl_conversion(np.array([6, 7, 8]), 2, num_robots, num_cells)
sys_prog |= {ltl_formula_1}
for i in range(num_robots):
    ltl_formula_2 = LTL_encoding.ltl_conversion_ap(np.array([2]), i, num_robots, num_cells)
    sys_prog |= {ltl_formula_2}
ltl_formula_3 = LTL_encoding.ltl_negate(LTL_encoding.cltl_conversion(np.array([1]), 1, num_robots, num_cells))
sys_safe |= {ltl_formula_3}


`omega.symbolic.symbolic` failed to import `dd.cudd`.
Will use `dd.autoref`.


In [2]:
# Create a GR(1) specification
specs = spec.GRSpec(env_vars, sys_vars, env_init, sys_init,
                    env_safe, sys_safe, env_prog, sys_prog)
specs.qinit = '\E \A'  # Moore initial condition synthesized too

In [3]:
print(env_vars)

set()


In [4]:
print(sys_vars)

{'loc': (0, 80)}


In [5]:
print(env_init)

set()


In [6]:
print(sys_init)

{'loc=27'}


In [7]:
print(env_safe)

set()


In [8]:
print(sys_safe)

{'loc=77 -> X (loc=47 || loc=49 || loc=65 || loc=67 || loc=71)', 'loc=27 -> X (loc=37 || loc=39 || loc=1 || loc=55 || loc=57)', 'loc=2 -> X (loc=14 || loc=28 || loc=32)', 'loc=38 -> X (loc=46 || loc=14 || loc=28 || loc=32 || loc=64 || loc=68)', 'loc=31 -> X (loc=41 || loc=37 || loc=43 || loc=5 || loc=1 || loc=3 || loc=7 || loc=59 || loc=55 || loc=57 || loc=61)', 'loc=76 -> X (loc=46 || loc=48 || loc=52 || loc=68 || loc=64 || loc=66)', 'loc=33 -> X (loc=43 || loc=39 || loc=7 || loc=3 || loc=61)', 'loc=42 -> X (loc=52 || loc=48 || loc=16 || loc=12 || loc=34 || loc=66)', 'loc=53 -> X (loc=23 || loc=25 || loc=41 || loc=43 || loc=79)', 'loc=4 -> X (loc=14 || loc=12 || loc=16 || loc=32 || loc=28 || loc=34)', 'loc=6 -> X (loc=16 || loc=12 || loc=34)', 'loc=29 -> X (loc=37 || loc=41 || loc=1 || loc=5 || loc=55 || loc=59)', 'loc=16 -> X (loc=26 || loc=22 || loc=24 || loc=8 || loc=4 || loc=6 || loc=44 || loc=42)', 'loc=15 -> X (loc=25 || loc=21 || loc=7 || loc=3 || loc=43 || loc=39)', 'loc=51 ->

In [9]:
print(env_prog)

set()


In [10]:
print(sys_prog)

{'( loc=65 || loc=2 || loc=38 || loc=74 || loc=11 || loc=47 || loc=56 || loc=29 )', '( loc=69 || loc=71 || loc=78 || loc=79 || loc=61 || loc=62 )', '( loc=18 || loc=19 || loc=21 || loc=22 || loc=23 || loc=24 || loc=25 || loc=26 )'}


In [3]:
ctrl = synth.synthesize(specs)
assert ctrl is not None, 'unrealizable'


# Generate a graphical representation of the controller for viewing
if not ctrl.save('gr1_set.png'):
    print(ctrl)

removed 0 nodes from 16 total


In [12]:
machines.random_run(ctrl, N=30)

move from
	 state: Sinit
	 with input:{}
	 to state: 0
	 reacting by producing output: {'loc': 27}
move from
	 state: 0
	 with input:{}
	 to state: 1
	 reacting by producing output: {'loc': 57}
move from
	 state: 1
	 with input:{}
	 to state: 2
	 reacting by producing output: {'loc': 67}
move from
	 state: 2
	 with input:{}
	 to state: 3
	 reacting by producing output: {'loc': 41}
move from
	 state: 3
	 with input:{}
	 to state: 4
	 reacting by producing output: {'loc': 65}
move from
	 state: 4
	 with input:{}
	 to state: 5
	 reacting by producing output: {'loc': 41}
move from
	 state: 5
	 with input:{}
	 to state: 6
	 reacting by producing output: {'loc': 71}
move from
	 state: 6
	 with input:{}
	 to state: 7
	 reacting by producing output: {'loc': 41}
move from
	 state: 7
	 with input:{}
	 to state: 8
	 reacting by producing output: {'loc': 53}
move from
	 state: 8
	 with input:{}
	 to state: 9
	 reacting by producing output: {'loc': 25}
move from
	 state: 9
	 with input:{}
	 to stat

([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  4,
  5],
 {'loc': [27,
   57,
   67,
   41,
   65,
   41,
   71,
   41,
   53,
   25,
   49,
   77,
   65,
   41,
   71,
   41,
   53,
   25,
   49,
   77,
   65,
   41,
   71,
   41,
   53,
   25,
   49,
   77,
   65,
   41]})

In [13]:
s = np.array([27,57,67,41,65,41,53,25,49,79,67,41,65,41,53,25,49,79,67,41,65,41,53,25,49,79,67,41,65,41])

In [14]:
states_2d_arr = State_encoding.array2coord(s, num_cells, num_robots, num_cols)

In [15]:
print(states_2d_arr)

[[[0, 0], [1, 0]], [[1, 0], [2, 0]], [[1, 1], [2, 1]], [[1, 2], [1, 1]], [[0, 2], [2, 1]], [[1, 2], [1, 1]], [[2, 2], [1, 2]], [[2, 1], [0, 2]], [[1, 1], [1, 2]], [[2, 1], [2, 2]], [[1, 1], [2, 1]], [[1, 2], [1, 1]], [[0, 2], [2, 1]], [[1, 2], [1, 1]], [[2, 2], [1, 2]], [[2, 1], [0, 2]], [[1, 1], [1, 2]], [[2, 1], [2, 2]], [[1, 1], [2, 1]], [[1, 2], [1, 1]], [[0, 2], [2, 1]], [[1, 2], [1, 1]], [[2, 2], [1, 2]], [[2, 1], [0, 2]], [[1, 1], [1, 2]], [[2, 1], [2, 2]], [[1, 1], [2, 1]], [[1, 2], [1, 1]], [[0, 2], [2, 1]], [[1, 2], [1, 1]]]
